In [50]:
#import required Libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import missingno as msno
from plotnine import *
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

**DATA CLEANING**

**Checking for Missing Values in all files**

In [51]:
#Load datasets
courses = pd.read_csv('~/finalproject/LHL-finalproject/datasets/courses.csv')
assessments = pd.read_csv('~/finalproject/LHL-finalproject/datasets/assessments.csv')
studentinfo = pd.read_csv('~/finalproject/LHL-finalproject/datasets/studentInfo.csv')
studentRegistration = pd.read_csv('~/finalproject/LHL-finalproject/datasets/studentRegistration.csv')
studentAssessment = pd.read_csv('~/finalproject/LHL-finalproject/datasets/studentAssessment.csv')

#Large files, couldn't be loaded to git
studentVle = pd.read_csv('~/finalproject/LHL-finalproject/datasets/studentvle.csv', nrows=999999)
vle = pd.read_csv('~/finalproject/LHL-finalproject/datasets/vle.csv')

**HANDLING MISSING VALUES**

**Courses data file**

In [52]:
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   code_module                 22 non-null     object
 1   code_presentation           22 non-null     object
 2   module_presentation_length  22 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 656.0+ bytes


In [53]:
# Check for missing values
missing_values = courses.isnull().sum()
print(missing_values)

code_module                   0
code_presentation             0
module_presentation_length    0
dtype: int64


There are no missing values in courses 

**Asessments data file**

In [54]:
assessments.replace(0.0, pd.NA, inplace=True)

In [55]:
assessments.replace('?', pd.NA, inplace=True)

In [56]:
#get a random sample of assessments to confirm presence of strange values
random_sample = assessments.sample(n=20, random_state=42)  
print(random_sample)


    code_module code_presentation  id_assessment assessment_type  date weight
15          BBB             2013B          14994             CMA   159    1.0
9           AAA             2014J           1761             TMA   166   20.0
201         GGG             2014J          37443             CMA   229   <NA>
82          DDD             2013B          25335             TMA    53   10.0
68          CCC             2014J          24291             TMA    32    9.0
97          DDD             2014B          25357             TMA    74   17.5
180         GGG             2013J          37422             CMA   229   <NA>
163         FFF             2014J          34904             CMA   241   <NA>
148         FFF             2013J          34877             TMA   173   25.0
183         GGG             2013J          37416             TMA   124   <NA>
101         DDD             2014B          25361            Exam   241  100.0
165         FFF             2014J          34906             CMA

Seems strange values like '?' have been taken care of

In [57]:
# Check for missing values
missing_values = assessments.isnull().sum()
print(missing_values)

code_module           0
code_presentation     0
id_assessment         0
assessment_type       0
date                 11
weight               56
dtype: int64


In [58]:
assessments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   code_module        206 non-null    object
 1   code_presentation  206 non-null    object
 2   id_assessment      206 non-null    int64 
 3   assessment_type    206 non-null    object
 4   date               195 non-null    object
 5   weight             150 non-null    object
dtypes: int64(1), object(5)
memory usage: 9.8+ KB


11 Final exams dates are missing
some courses have weights missing for assignments

**Student Assessment**

In [59]:
#get a random sample of assessments to confirm presence of strange values
random_sample = studentAssessment.sample(n=5, random_state=42)  
print(random_sample)

        id_assessment  id_student  date_submitted  is_banked score
58822           24295      335914              21          0    78
158625          34910      648903             235          0    65
81731           25353      103800             207          0    41
68200           25337      537926             116          0    15
160473          37417      575559             171          0    80


the is_banked column is mostly empty, drop column.

In [60]:
studentAssessment.replace(0.0, pd.NA, inplace=True)
studentAssessment.shape

(173912, 5)

In [61]:
# Check for missing values in student assessments
missing_values = studentAssessment.isnull().sum()
print(missing_values)

id_assessment          0
id_student             0
date_submitted        61
is_banked         172003
score                  0
dtype: int64


In [62]:
#get a random sample to confirm presence of strange values
random_sample = studentAssessment.sample(n=5, random_state=42)  
print(random_sample)


        id_assessment  id_student date_submitted is_banked score
58822           24295      335914             21      <NA>    78
158625          34910      648903            235      <NA>    65
81731           25353      103800            207      <NA>    41
68200           25337      537926            116      <NA>    15
160473          37417      575559            171      <NA>    80


In [63]:
# Check if there are any occurrences of 0.0 in the 'score' column
zero_occurrences = (studentAssessment['score'] == 0.0).any()

if zero_occurrences:
    print("There are occurrences of 0.0 in the 'score' column.")
else:
    print("There are no occurrences of 0.0 in the 'score' column.")


There are no occurrences of 0.0 in the 'score' column.


**studentinfo data file**

In [64]:
studentinfo.replace(0.0, pd.NA, inplace=True)
studentinfo.replace('?', pd.NA, inplace=True)
studentinfo.shape

(32593, 12)

In [65]:
studentinfo.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,<NA>,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,<NA>,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,<NA>,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,<NA>,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,<NA>,60,N,Pass


In [66]:
# Check for missing values
missing_values = studentinfo.isnull().sum()
print(missing_values)

code_module                 0
code_presentation           0
id_student                  0
gender                      0
region                      0
highest_education           0
imd_band                 1111
age_band                    0
num_of_prev_attempts    28421
studied_credits             0
disability                  0
final_result                0
dtype: int64


In [67]:
#get a random sample to confirm presence of strange values
random_sample = studentinfo.sample(n=5, random_state=42)  
print(random_sample)


      code_module code_presentation  id_student gender                region  \
21105         EEE             2014B     2632165      M                 Wales   
30047         FFF             2014J     2678580      M  North Western Region   
7302          BBB             2014J      642122      F              Scotland   
12840         CCC             2014J     1101190      M  North Western Region   
29147         FFF             2014J      656579      F     South West Region   

           highest_education imd_band age_band num_of_prev_attempts  \
21105  A Level or Equivalent   70-80%     0-35                 <NA>   
30047  A Level or Equivalent   60-70%     0-35                 <NA>   
7302        HE Qualification   20-30%    35-55                 <NA>   
12840       HE Qualification   30-40%    35-55                 <NA>   
29147  A Level or Equivalent   50-60%     0-35                 <NA>   

       studied_credits disability final_result  
21105               60          N         P

In [68]:
studentinfo['imd_band'].isnull().shape[0]

32593

**vle data file**

In [69]:
vle.replace(0.0, pd.NA, inplace=True)
vle.replace('?', pd.NA, inplace=True)
vle.shape

(6364, 6)

In [70]:
vle.head()

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,<NA>,<NA>
1,546712,AAA,2013J,oucontent,<NA>,<NA>
2,546998,AAA,2013J,resource,<NA>,<NA>
3,546888,AAA,2013J,url,<NA>,<NA>
4,547035,AAA,2013J,resource,<NA>,<NA>


this shows missing data in the week_from and week_to columns


week_from and week_to tell us the timeframe certain contents are supposed to be used. Unfortunately, over 82% of this content is missing from the dataset.

In [71]:
# Check for missing values
missing_values = vle.isnull().sum()
print(missing_values)

id_site                 0
code_module             0
code_presentation       0
activity_type           0
week_from            5243
week_to              5243
dtype: int64


 **studentRegistration data file**

In [72]:
studentRegistration.replace(0.0, pd.NA, inplace=True)
studentRegistration.replace('?', pd.NA, inplace=True)
studentRegistration.shape

(32593, 5)

In [73]:
studentRegistration.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159,<NA>
1,AAA,2013J,28400,-53,<NA>
2,AAA,2013J,30268,-92,12
3,AAA,2013J,31604,-52,<NA>
4,AAA,2013J,32885,-176,<NA>


In [74]:
# Check for missing values
missing_values = studentRegistration.isnull().sum()
print(missing_values)

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64


There are a decent amount of students who withdrew from a course before it started. We will remove these students from our dataset later, since we don't have any academic performance data for them.

**Studentvle file**

In [75]:
studentVle.replace(0.0, pd.NA, inplace=True)
studentVle.replace('?', pd.NA, inplace=True)
studentVle.shape

(999999, 6)

In [76]:
# Check for missing values
missing_values = studentVle.isnull().sum()
print(missing_values)

code_module             0
code_presentation       0
id_student              0
id_site                 0
date                 7770
sum_click               0
dtype: int64


In [77]:
random_sample = studentVle.sample(n=5, random_state=42)  

print(random_sample)


       code_module code_presentation  id_student  id_site date  sum_click
987230         BBB             2013J      301987   703732   97          6
79954          AAA             2013J     1765788   546732   80         17
567130         BBB             2013B      446856   542864   80          3
500891         BBB             2013B      108589   543020   44          2
55399          AAA             2013J     1626710   547000   42          1


**HANDLING MISSING VALUES**

Create seven lists, each containing the column names of different DataFrames.

In [78]:
set1 = list(assessments.columns.values)
set2 = list(courses.columns.values)
set3 = list(studentAssessment.columns.values)
set4 = list(studentinfo.columns.values)
set5 = list(studentRegistration.columns.values)
set6 = list(studentVle.columns.values)
set7 = list(vle.columns.values)

In [79]:
all_columns = [set1, set2, set3, set4, set5, set6, set7]
columns_count = [assessments.shape,courses.shape,studentAssessment.shape, studentinfo.shape, studentRegistration.shape, studentVle.shape, vle.shape]
columns_header = ['assessments', 'courses', 'studentAssessment', 'studentInfo', 'studentRegistration', 'studentVle', 'vle' ]

Creates a dictionary named d with three keys: 
'Table Name', 
'Rows, Columns', and 
'Column Names'.  As defined above.

In [80]:
d = {'Table Name':columns_header,'Rows, Columns': columns_count,'Column Names':all_columns}

In [81]:
df = pd.set_option('max_colwidth', 200)
df = pd.DataFrame(d)

In [82]:
df

,Table Name,"Rows, Columns",Column Names
0,assessments,"(206, 6)","[code_module, code_presentation, id_assessment, assessment_type, date, weight]"
1,courses,"(22, 3)","[code_module, code_presentation, module_presentation_length]"
2,studentAssessment,"(173912, 5)","[id_assessment, id_student, date_submitted, is_banked, score]"
3,studentInfo,"(32593, 12)","[code_module, code_presentation, id_student, gender, region, highest_education, imd_band, age_band, num_of_prev_attempts, studied_credits, disability, final_result]"
4,studentRegistration,"(32593, 5)","[code_module, code_presentation, id_student, date_registration, date_unregistration]"
5,studentVle,"(999999, 6)","[code_module, code_presentation, id_student, id_site, date, sum_click]"
6,vle,"(6364, 6)","[id_site, code_module, code_presentation, activity_type, week_from, week_to]"


Remove all missing values

In [83]:
# Dropping all the missing values
assessments.dropna(inplace=True)
courses.dropna(inplace=True)
studentAssessment.dropna(inplace=True)
studentinfo.dropna(inplace=True)
studentRegistration.dropna(inplace=True)
studentVle.dropna(inplace=True)
vle.dropna(inplace=True)

In [84]:
columns_count = [assessments.shape,courses.shape,studentAssessment.shape, studentinfo.shape, studentRegistration.shape, studentVle.shape, vle.shape]
d = {'Table Name':columns_header,'Rows, Columns': columns_count,'Column Names':all_columns}
df = pd.set_option('max_colwidth', 200)
df = pd.DataFrame(d)

In [85]:
df

,Table Name,"Rows, Columns",Column Names
0,assessments,"(139, 6)","[code_module, code_presentation, id_assessment, assessment_type, date, weight]"
1,courses,"(22, 3)","[code_module, code_presentation, module_presentation_length]"
2,studentAssessment,"(1909, 5)","[id_assessment, id_student, date_submitted, is_banked, score]"
3,studentInfo,"(4051, 12)","[code_module, code_presentation, id_student, gender, region, highest_education, imd_band, age_band, num_of_prev_attempts, studied_credits, disability, final_result]"
4,studentRegistration,"(10033, 5)","[code_module, code_presentation, id_student, date_registration, date_unregistration]"
5,studentVle,"(992229, 6)","[code_module, code_presentation, id_student, id_site, date, sum_click]"
6,vle,"(1121, 6)","[id_site, code_module, code_presentation, activity_type, week_from, week_to]"


For removing missing values, there are chnages in mumber of rows

**MERGING DATAFRAMES**

In [86]:
from functools import reduce
dfs = [studentAssessment, studentinfo, studentRegistration]
df_final = reduce(lambda left,right: pd.merge(left,right,on='id_student'), dfs)
df_final['final_result'].value_counts()

final_result
Withdrawn      1151
Fail            669
Pass            459
Distinction      47
Name: count, dtype: int64

In [87]:
df_final

,id_assessment,id_student,date_submitted,is_banked,score,code_module_x,code_presentation_x,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,code_module_y,code_presentation_y,date_registration,date_unregistration
0,1758,1970390,-1,1,78,AAA,2014J,M,West Midlands Region,Lower Than A Level,0-10%,35-55,1,120,N,Pass,AAA,2013J,-108,240
1,1759,1970390,-1,1,90,AAA,2014J,M,West Midlands Region,Lower Than A Level,0-10%,35-55,1,120,N,Pass,AAA,2013J,-108,240
2,1760,1970390,-1,1,90,AAA,2014J,M,West Midlands Region,Lower Than A Level,0-10%,35-55,1,120,N,Pass,AAA,2013J,-108,240
3,1761,1970390,-1,1,80,AAA,2014J,M,West Midlands Region,Lower Than A Level,0-10%,35-55,1,120,N,Pass,AAA,2013J,-108,240
4,1762,1970390,-1,1,80,AAA,2014J,M,West Midlands Region,Lower Than A Level,0-10%,35-55,1,120,N,Pass,AAA,2013J,-108,240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,37437,631446,-1,1,64,GGG,2014J,F,East Anglian Region,HE Qualification,90-100%,35-55,1,30,N,Pass,GGG,2014B,-24,219
2322,37438,631446,-1,1,100,GGG,2014J,F,East Anglian Region,HE Qualification,90-100%,35-55,1,30,N,Pass,GGG,2014B,-24,219
2323,37439,631446,-1,1,100,GGG,2014J,F,East Anglian Region,HE Qualification,90-100%,35-55,1,30,N,Pass,GGG,2014B,-24,219
2324,37440,631446,-1,1,100,GGG,2014J,F,East Anglian Region,HE Qualification,90-100%,35-55,1,30,N,Pass,GGG,2014B,-24,219


In [88]:
df_final = df_final.drop(['date_registration','date_unregistration', 'id_assessment', 'id_student', 'date_submitted', 'score', 'studied_credits'],axis =1) # too many NaN values

In [89]:
df_final.dropna(inplace=True)
df_final['final_result'].value_counts()

final_result
Withdrawn      1151
Fail            669
Pass            459
Distinction      47
Name: count, dtype: int64

In [90]:
#Keep a copy of the merged dataframe
df_final_original = df_final

In [91]:
df_final_original.head()

,is_banked,code_module_x,code_presentation_x,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,disability,final_result,code_module_y,code_presentation_y
0,1,AAA,2014J,M,West Midlands Region,Lower Than A Level,0-10%,35-55,1,N,Pass,AAA,2013J
1,1,AAA,2014J,M,West Midlands Region,Lower Than A Level,0-10%,35-55,1,N,Pass,AAA,2013J
2,1,AAA,2014J,M,West Midlands Region,Lower Than A Level,0-10%,35-55,1,N,Pass,AAA,2013J
3,1,AAA,2014J,M,West Midlands Region,Lower Than A Level,0-10%,35-55,1,N,Pass,AAA,2013J
4,1,AAA,2014J,M,West Midlands Region,Lower Than A Level,0-10%,35-55,1,N,Pass,AAA,2013J


In [92]:
df_final_original.shape

(2326, 13)

In [93]:
# converting the final table to labelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_final = df_final.apply(le.fit_transform)

In [94]:
df_final

,is_banked,code_module_x,code_presentation_x,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,disability,final_result,code_module_y,code_presentation_y
0,0,0,3,1,11,2,0,1,0,0,2,0,1
1,0,0,3,1,11,2,0,1,0,0,2,0,1
2,0,0,3,1,11,2,0,1,0,0,2,0,1
3,0,0,3,1,11,2,0,1,0,0,2,0,1
4,0,0,3,1,11,2,0,1,0,0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,0,6,3,0,0,1,9,1,0,0,2,6,2
2322,0,6,3,0,0,1,9,1,0,0,2,6,2
2323,0,6,3,0,0,1,9,1,0,0,2,6,2
2324,0,6,3,0,0,1,9,1,0,0,2,6,2


In [96]:
# dropping disability (non significant)
df_final = df_final.drop(['disability'],axis =1)
df_final_original = df_final_original.drop(['disability'],axis =1)

In [98]:
# Assuming df is your DataFrame
df_final.to_csv('~/finalproject/LHL-finalProject/preprocessed_data/dataencoded.csv', index=False)
df_final_original.to_csv('~/finalproject/LHL-finalProject/preprocessed_data/data.csv', index=False)
